In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql
import re
import time
from datetime import datetime
import os
import logging
import sshtunnel
from sshtunnel import SSHTunnelForwarder
import paramiko


user_name='trishala/'

ssh_host = '10.120.1.50'
ssh_username = 'research-user'
localhost = '127.0.0.1'

# prod
ezcapc_user = 'read_only'
ezcapc_pwd = 'WHC63fKyjudfSz2U'
ezcapc_host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
ezcapc_db_name = 'CAPC_APIGATEWAY'
env='production'
paas_user = 'read_only'
paas_pwd = 'eUN2cORh1sCRG998'
paas_host = 'paas-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
paas_db_name = 'FACILITY_SERVICE'

# # stag
# ezcapc_user = 'read_only'
# ezcapc_pwd = '3j4A3PaQc8BV050R'
# ezcapc_host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# ezcapc_db_name = 'CAPC_APIGATEWAY'
# env='staging'
# paas_user = 'read_only'
# paas_pwd = 's9a843QvgkPje5SD'
# paas_host = 'paas-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# paas_db_name = 'METADATA_SERVICE'



# ezcapc_db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(ezcapc_user,ezcapc_pwd,ezcapc_host,ezcapc_db_name)
# ezcapc_db_connection = create_engine(ezcapc_db_connection_str)

# paas_db_connection_str = 'mysql+pymysql://{}:{}@{}'.format(paas_user,paas_pwd,paas_host)
# paas_db_connection = create_engine(paas_db_connection_str)

In [ ]:
def open_ssh_tunnel(verbose=False):
    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
    
    global ezcapc_tunnel
    global paas_tunnel
    ezcapc_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/trishalaswain/pem/research_user.pem',
        remote_bind_address = (ezcapc_host, 3306)
    )
    paas_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/trishalaswain/pem/research_user.pem',
        remote_bind_address = (paas_host, 3306)
    )
    ezcapc_tunnel.start()
    paas_tunnel.start()
    print(ezcapc_tunnel.is_active)
    print(paas_tunnel.is_active)

In [ ]:
def mysql_connect():
    global ezcapc_connection
    global paas_connection
    ezcapc_connection = pymysql.connect(
        host=localhost,
        user=ezcapc_user,
        passwd=ezcapc_pwd,
        database=ezcapc_db_name,
        port=ezcapc_tunnel.local_bind_port
    )

    paas_connection = pymysql.connect(
        host=localhost,
        user=paas_user,
        passwd=paas_pwd,
        database=paas_db_name,
        port=paas_tunnel.local_bind_port
    )

In [ ]:
open_ssh_tunnel()
mysql_connect()

In [ ]:
# please change esm.last_coding_date & am.facility_id = '314' in the query as per requirement
# am.facility_id is prod_id from go-live sheet
query = '''
SELECT 
    am.account_number,
    dpd.nlp_request_s3_path,
    dpd.nlp_ner_entities_result_s3_path,
    am.facility_id,
    dm.service_date,
    dpd.nlp_pcs_xml_result_s3_path,
    dpd.preprocessing_request_s3_path,
    dpd.preprocessing_result_s3_path,
    dm.orignal_path,
    dm.type_id,
    dpd.created_date,
    dpd.ocr_request_s3_path,
    dpd.ocr_result_s3_path,
    dpd.eandmcs_request_s3_path,
    dpd.eandmcs_result_s3_path,
    dpd.ruleengine_request_s3_path,
    dpd.cptcs_request_s3_path,
    dpd.cptcs_result_s3_path,
    dm.account_id,
    em.id as encounter_id,
    dc.code AS suggested_code,
    dpd.id AS DPD_ID,
    dm.path as filePath,
    eandm.code AS final_code,
    CONVERT_TZ(em.service_date, 'UTC', 'US/Central') AS service_date,
    GROUP_CONCAT(DISTINCT eandm.code) AS final,
    GROUP_CONCAT(DISTINCT dc.code) AS suggested,
    -- am.account_number AS account_number,
    facility_id,
    client_id
FROM
    account_mst am
        INNER JOIN
    document_mst dm ON dm.account_id = am.id
        AND dm.is_physician_document = 1
        AND dm.is_active = 1
        INNER JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    document_code dc ON dc.document_id = dm.id
        AND dc.is_active = 1
        AND dc.code_category = 'EANDM'
        INNER JOIN
    encounter_mst em ON em.account_id = dm.account_id
        AND em.service_date = dm.service_date
        AND em.physician_id = dm.physician_id
        AND em.is_active = 1
        INNER JOIN
    encounter_status_map esm ON esm.encounter_id = em.id
        AND esm.is_active = 1
        AND esm.encounter_status_id IN (9 , 17)
        INNER JOIN
    eandm_code eandm ON eandm.encounter_id = em.id
        AND eandm.process_id = esm.process_id
        AND eandm.user_id = esm.user_id
        AND eandm.is_active = 1
	
WHERE
dpd.id = (SELECT 
            id
        FROM
            document_processing_detail
        WHERE
            document_id = dm.id
        ORDER BY created_date DESC
        LIMIT 1)
        AND am.facility_id = '250'
        AND am.client_id = '15'
        AND esm.id = (SELECT 
            id
        FROM
            encounter_status_map
        WHERE
            encounter_id = em.id
                AND encounter_status_id IN (9 , 17)
        ORDER BY last_coding_date DESC
        LIMIT 1)
        AND am.is_active = 1
        AND CONVERT_TZ(esm.last_coding_date,
            'UTC',
            'US/Central') >= '2023-08-27'

GROUP BY am.account_number , em.physician_id , em.service_date , dm.id
'''
        # AND CONVERT_TZ(em.service_date,
        #     'UTC',
        #     'US/Central') > '2023-03-15'
        
        #     -- am.account_number = 8594135
        
            #     AND CONVERT_TZ(em.service_date,
            # 'UTC',
            # 'US/Central') > '2023-07-01'

In [ ]:
df = pd.read_sql(query, con=ezcapc_connection)

In [ ]:
df

In [ ]:
df = df[['service_date', 'account_number', 'suggested', 'final']]

In [ ]:
df.final.value_counts()

In [ ]:
# df.to_excel('/home/trishalaswain/Documents/AGS_PROJECTS/OCR_SERVICE/neighbourhood_testing/ACCURACY_MONITORING/493937_BILLED.xlsx', index=False)

In [ ]:
df[df.final.str.contains('99281,99283,99284')]

In [ ]:
# check for nan in final and suggested columns

In [ ]:
df = df[~df.final.str.contains(',')]

In [ ]:
def get_counts(x):
    
    suggested = str(x['suggested'])
    final = str(x['final'])
    
    suggested_set = set(suggested.split(','))   
    final_set = set(final.split(','))
    
    accepted = len(suggested_set.intersection(final_set))
    rejected = len(suggested_set - final_set)
    added = len(final_set - suggested_set)
    if (rejected >= 1 or added >= 1):
        print('rejected',rejected)
        print('added',added)
        print('suggested_set',suggested_set,'\t final_set',final_set)
        print('accountNumber', str(x['account_number']))
            
    return pd.Series({'accepted':accepted, 'rejected':rejected, 'added':added})

In [ ]:
count_df =  df.apply(get_counts, axis=1)

In [ ]:
count_df

In [ ]:
count_df.accepted.sum(), count_df.rejected.sum(), count_df.added.sum()

In [ ]:
precision = count_df.accepted.sum() / (count_df.accepted.sum() + count_df.rejected.sum())

In [ ]:
recall = count_df.accepted.sum() / (count_df.accepted.sum() + count_df.added.sum())

In [ ]:
f1_score = (2 * precision * recall) / (recall + precision)

In [ ]:
precision, recall, f1_score